In [6]:
from openai import OpenAI
import openai
import backoff
import os



#OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

#client = OpenAI(os.getenv('OPENAI_API_KEY')) pilotblogger

client = OpenAI(os.getenv('OPENAI_API_KEY')) #shiprocker 



response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [ ]:
import anthropic

clienta = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
)


message = clienta.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "How are you today?"}
    ]
)

print(message.content)# Assuming text_block_list is the list containing the TextBlock object
text_block = message.content[0]  # Access the TextBlock from the list
text = text_block.text  # Get the text from the TextBlock
print(text)  # Print the text



In [ ]:
response.choices[0].message.content

In [ ]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None,  # whether to return log probabilities of the output tokens or not. If true, returns the log probabilities of each output token returned in the content of message..
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": logprobs,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [ ]:
CLASSIFICATION_PROMPT = """You will be given a headline of a news article.
Classify the article into one of the following categories: Technology, Politics, Sports, and Art.
Return only the name of the category, and nothing else.
MAKE SURE your output is one of the four categories stated.
Article headline: {headline}"""

In [ ]:
headlines = [
    "Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.",
    "Local Mayor Launches Initiative to Enhance Urban Public Transport.",
    "Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut",
]

In [ ]:
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4",
    )
    print(f"Category: {API_RESPONSE.choices[0].message.content}\n")

In [69]:
import numpy as np
from IPython.display import HTML
for headline in headlines:
    print(f"\nHeadline: {headline}")
    API_RESPONSE = get_completion(
        [{"role": "user", "content": CLASSIFICATION_PROMPT.format(headline=headline)}],
        model="gpt-4",
        logprobs=True,
        top_logprobs=2,
    )
    top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
    html_content = ""
    for i, logprob in enumerate(top_two_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )
    display(HTML(html_content))
    print("\n")


Headline: Tech Giant Unveils Latest Smartphone Model with Advanced Photo-Editing Features.





Headline: Local Mayor Launches Initiative to Enhance Urban Public Transport.





Headline: Tennis Champion Showcases Hidden Talents in Symphony Orchestra Debut


In [2]:
import openai
import os

from openai import OpenAI
import openai
import backoff
import os
import numpy as np



#OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY')) #ship

# Setup the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')  # Ensure you have this set in your environment variables

def generate_story_passage(context):
    """ Generate a story passage based on the given context using OpenAI's API. """
    response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "Generate a story passage based on the given context"},
        {"role": "user", "content": context}]
    )   
    return response.choices[0].message.content.strip()

def score_passage(passage, context):
    """ Score the generated passage for coherence, relevance, and commentary. """
    scoring_prompt = f"Review the following passage: {passage} Based on the initial context: {context}\n\n"
    scoring_prompt += "Will the story have a happy outcome? YOU MUST ANSWER WITH ONLY EITHER yes' or 'no'.\n\n"


    response = client.chat.completions.create(
    model="gpt-4-turbo",
     logprobs=True,
    top_logprobs=2,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": scoring_prompt}]
    )   

    return response.choices[0].message.content.strip(), response.choices[0].logprobs

# Example usage
if __name__ == "__main__":
    context = "The old library held secrets beyond the books on its shelves. The protagonist uncovers a hidden diary behind the ancient texts. a sad end will befall the protagonist."
    passage = generate_story_passage(context)
    print("Generated Passage:", passage)

    score, log_probs = score_passage(passage, context)
    print("Scoring Results:", score)
    print("Log Probabilities:", log_probs)

    top_two_logprobs = log_probs.content[0].top_logprobs
    html_content = ""
    for i, logprob in enumerate(top_two_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )
    display(HTML(html_content))
    print("\n")


Generated Passage: In the dimly lit corner of the old library, nestled between the musty, leather-bound volumes of forgotten lore, Helena discovered a small, inconspicuous diary. Its cover was worn, the leather cracked and faded with age, blending almost seamlessly with the row of antique books on the shelf. She had been volunteering in the library, helping the elderly librarian, Mr. Pembroke, catalog its extensive collection. The library had always felt like a sanctuary to her, the silence filled with the whispers of ages past.

Mr. Pembroke had warned her about the west wing of the library. "Don't stray too far into the old collections," he would say, his eyes clouding over with a trace of unexplained fear. But Helena, with a curiosity as fierce as the winds on the coast outside, couldn't help herself. It was during one such forbidden expedition that she stumbled upon the hidden diary.

The diary seemed to beckon her, its pages a gateway to another time. She opened it gently, the spi

NameError: name 'HTML' is not defined

In [3]:
log_probs

ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='No', bytes=[78, 111], logprob=-0.04050042, top_logprobs=[TopLogprob(token='No', bytes=[78, 111], logprob=-0.04050042), TopLogprob(token='no', bytes=[110, 111], logprob=-3.2280004)]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=-0.009775686, top_logprobs=[TopLogprob(token='.', bytes=[46], logprob=-0.009775686), TopLogprob(token='<|end|>', bytes=None, logprob=-4.6347756)])])

In [4]:
top_two_logprobs 

[TopLogprob(token='No', bytes=[78, 111], logprob=-0.04050042),
 TopLogprob(token='no', bytes=[110, 111], logprob=-3.2280004)]

In [84]:
print("Log Probabilities:", log_probs)

Log Probabilities: ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-3.094816e-05, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-3.094816e-05), TopLogprob(token='No', bytes=[78, 111], logprob=-11.0625305)]), ChatCompletionTokenLogprob(token='.', bytes=[46], logprob=-0.46241638, top_logprobs=[TopLogprob(token='.', bytes=[46], logprob=-0.46241638), TopLogprob(token='<|end|>', bytes=None, logprob=-0.9936664)])])


In [87]:
log_probs.content[0].top_logprobs

[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-3.094816e-05),
 TopLogprob(token='No', bytes=[78, 111], logprob=-11.0625305)]

In [ ]:
response.choices[0].message.content.strip()